In [72]:
import pandas as pd
import numpy as np
from scipy import spatial
from gensim.models import word2vec
import pickle

vector_size = 250

In [73]:
'''
csv読み込み
store_idをindexに変更
'''

df = pd.read_csv('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/csv/review_wakati.csv')
df = df.drop(['store_id'],axis=1)
df = df.rename(columns={'Unnamed: 0':'store_id'}).set_index('store_id')

In [74]:
'''
pickleで学習済みreview達のvector読み出し
'''

f = open("../pickle/skip_list.txt","rb")
skip_list = pickle.load(f)
f.close()

In [75]:
'''
学習済みモデルの読み出し
'''

model = word2vec.Word2Vec.load('/Users/kazuki_takahashi/Desktop/python_study/datamix_ramen/models/skip_w2v.model')

In [76]:
'''
ここまで準備
'''

'\nここまで準備\n'

In [77]:
def sentence_similarity(vec1, vec2):
    '''
    二つの文章のコサイン類似度を返す
    '''
    return 1 - spatial.distance.cosine(vec1, vec2)

In [78]:
'''
store_idをkeyに指定文書と各行のreviewのコサイン類似度をスコアとして入れる
'''

result = {}
keyword = avg_feature_vectors(['カレー', '極太'], model)
for i, row in df.iterrows():
    score = sentence_similarity(keyword, skip_list[i])
    result[i] = score * row.score


/Users/kazuki_takahashi/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [79]:
'''
scoreの高い順にsort
'''

score_sorted = sorted(result.items(), key=lambda x:x[1], reverse=True)

In [80]:
'''
score高い10件を書き出し
'''

for t in score_sorted[:10]:
    row = df[df.index == t[0]]
    name, score, station = row.store_name.values[0], row.score.values[0], row.station.values[0]
    print('店名:{}, 食べログスコア:{}, 独自スコア:{:.2f}, 最寄駅:{}'.format(name,score,t[1]*100,station))
    

店名:しゅういち 恵比寿店, 食べログスコア:3.54, 独自スコア:227.13, 最寄駅:恵比寿
店名:インディアン 本店, 食べログスコア:3.68, 独自スコア:227.01, 最寄駅:蓮沼
店名:徳川膳武, 食べログスコア:3.53, 独自スコア:225.13, 最寄駅:東新宿
店名:麺屋虎杖 有楽町, 食べログスコア:3.57, 独自スコア:224.64, 最寄駅:有楽町
店名:インディアン 池上店, 食べログスコア:3.53, 独自スコア:224.40, 最寄駅:池上
店名:お茶の水、大勝軒 BRANCHING, 食べログスコア:3.52, 独自スコア:221.59, 最寄駅:九段下
店名:とろ肉つけ麺 魚とん, 食べログスコア:3.64, 独自スコア:219.54, 最寄駅:小川町
店名:スパイス・ラー麺 卍力 西葛西店, 食べログスコア:3.79, 独自スコア:216.84, 最寄駅:西葛西
店名:お茶の水 大勝軒, 食べログスコア:3.66, 独自スコア:216.41, 最寄駅:神保町
店名:熊公, 食べログスコア:3.56, 独自スコア:212.51, 最寄駅:浅草橋
